# Qual curso tem menor desvio padrão de notas ?

In [24]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, MetaData, Table, inspect 

In [25]:
engine = create_engine('sqlite:///:memory:') # Criando banco SQL
engine 

Engine(sqlite:///:memory:)

In [26]:
tabela_notas = pd.read_excel('files/tabela_nota.xlsx') # Importando tabela de notas
tabela_notas.drop(tabela_notas.loc[tabela_notas['id_materia']==0].index, inplace=True) # Não é possivel saber qual materia ele fez
tabela_notas['id'] = range(tabela_notas.shape[0])
tabela_notas.set_index('id', inplace=True)
tabela_notas.index = tabela_notas.index + 1
tabela_notas.rename(columns={'id_professor(a)': 'id_professor'}, inplace=True)
tabela_notas.to_sql('notas', engine) 
inspector = inspect(engine) 
print(inspector.get_table_names())

['notas']


In [65]:
tabela_estudantes = pd.read_excel('files/tabela_estudante.xlsx', index_col=False) # Importando tabela de alunos
tabela_estudantes.set_index('Unnamed: 0', inplace=True)
tabela_estudantes.index.rename('id', inplace=True)

In [59]:
# Buscando todos os alunos do curso de um determinado curso
def busca_estudantes(curso: str):
    estudantes = tabela_estudantes[tabela_estudantes['curso'] == curso]
    return estudantes


# Preenchendo lista com todas as notas do curso
def preenchendo_lista_notas(estudantes):
    lista_notas = []
    
    for i in range(estudantes.shape[0]):
        query = f'select * from notas where id_estudante = {estudantes.index[i]}'
        notas = pd.read_sql(query, engine)
        lista_notas.append(notas['nota'].tolist())
        
    return lista_notas

# Pega a lista de listas e transforma em um vetor com os valores
def transforma_list(tamanho_notas, lista_notas):
    lista_ = []
    
    for i in range(tamanho_notas):
        for j in range(len(lista_notas[i])):
            lista_.append(lista_notas[i][j])
    
    return lista_

# Exportar no excel
def exporta_para_excel(itens: list, nome_coluna_itens: str, valores: list, nome_coluna_valores: str, caminho: str):
    dados = pd.DataFrame({nome_coluna_itens: itens, nome_coluna_valores: valores})
    dados.to_excel(caminho, index=False)

## Estatística

In [60]:
estudantes_estatistica = busca_estudantes('Estatística')
lista_notas_estatistica = []
lista_notas_estatistica = preenchendo_lista_notas(estudantes_estatistica)
lista_notas_estatistica = transforma_df_para_list(6, lista_notas_estatistica)
desvio_padrao_estatistica = np.std(lista_notas_estatistica)
desvio_padrao_estatistica

3.153866674417294

## Matemática Aplicada

In [61]:
estudantes_matematica_aplicada = busca_estudantes('Matemática Aplicada')
lista_notas_matematica_aplicada = []
lista_notas_matematica_aplicada = preenchendo_lista_notas(estudantes_matematica_aplicada)
lista_notas_matematica_aplicada = transforma_df_para_list(6, lista_notas_matematica_aplicada)
desvio_padrao_matematica_aplicada = np.std(lista_notas_matematica_aplicada)
desvio_padrao_matematica_aplicada

2.9765752132274432

In [64]:
exporta_para_excel(cursos, 'Curso', desvio_padrao_de_notas, 'Desvio Padrão de Notas', 'files/files_questions/question_5/desvio_padrao_nota_curso.xlsx')